(lightning-trainer-exp-tracking)=

# Using W&B, CometML, and MLFlow Logger in LightningTrainer

W&B, CometML, and MLFlow are all popular tools in the field of machine learning for managing, visualizing, and tracking experiments. 

PyTorch Lightning provides built-in support for logging metrics to W&B and CometML during training, allowing users to easily track their model performance over time. MLFlow can also be used to track experiments and store artifacts such as trained models and hyperparameters. With AIR LightningTrainer, you can still leverage the built-in supports with the PyTorch Lightning's `logger`.


## Define your model and dataloader

No need for any code change here. We simply create a dummy model with dummy datasets for demo.

In [9]:
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader

# create dummy data
X = torch.randn(128, 3)  # 128 samples, 3 features
y = torch.randint(0, 2, (128,))  # 128 binary labels

# create a TensorDataset to wrap the data
dataset = TensorDataset(X, y)

# create a DataLoader to iterate over the dataset
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [10]:
# Define a dummy model
class DummyModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer = torch.nn.Linear(3, 1)
    
    def forward(self, x):
        return self.layer(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy_with_logits(y_hat.flatten(), y.float())

        # The metrics below will be reported to Loggers
        self.log('train_loss', loss)
        self.log_dict({"metric_1": batch_idx * 10, "metric_2": batch_idx * 100})
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)
    

## Define your loggers

You don't need to make any changes to your original code for offline logging. If you would like to upload your log online to W&B or CometML, make sure to set up the API key environment variables on the head node.

In [11]:
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.loggers.comet import CometLogger
from pytorch_lightning.loggers.mlflow import MLFlowLogger
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.utilities.rank_zero import rank_zero_only
import wandb

# A callback to login wandb in each worker
class WandbLoginCallback(pl.Callback):
    def __init__(self, key):
        self.key = key

    def setup(self, trainer, pl_module, stage) -> None:
        wandb.login(key=self.key)


def create_loggers(name, project_name, save_dir="./logs", offline=False):
    # Wandb
    wandb_api_key = os.environ.get("WANDB_API_KEY", None)

    # Avoid creating a new experiment run in the driver.
    rank_zero_only.rank = None
    wandb_logger = WandbLogger(
        name=name, project=project_name, save_dir=f"{save_dir}/wandb", offline=offline
    )
    callbacks = [] if offline else [WandbLoginCallback(key=wandb_api_key)]

    # CometML
    comet_api_key = os.environ.get("COMET_API_KEY", None)
    comet_logger = CometLogger(
        api_key=comet_api_key,
        experiment_name=name,
        project_name=project_name,
        save_dir=f"{save_dir}/comet",
        offline=offline,
    )

    # MLFlow
    mlflow_logger = MLFlowLogger(
        run_name=name, experiment_name=project_name, save_dir=f"{save_dir}/mlflow"
    )

    # Tensorboard
    tensorboard_logger = TensorBoardLogger(
        name=name, save_dir=f"{save_dir}/tensorboard"
    )

    return [wandb_logger, comet_logger, mlflow_logger, tensorboard_logger], callbacks


loggers, callbacks = create_loggers(
    name="demo-run", project_name="demo-project", offline=False
)


CometLogger will be initialized in online mode


In [12]:
# # FOR SMOKE TESTS
loggers, callbacks = create_loggers(
    name="demo-run", project_name="demo-project", offline=True
)


## Train the model and check out your logs:

In [18]:
from ray.train.lightning import LightningConfigBuilder, LightningTrainer

builder = LightningConfigBuilder()
builder.module(cls=DummyModel)
builder.trainer(max_epochs=5, accelerator="cpu", logger=loggers, callbacks=callbacks, log_every_n_steps=1)
builder.fit_params(train_dataloaders=dataloader)
lightning_config = builder.build()


In [19]:
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig

scaling_config = ScalingConfig(num_workers=4, use_gpu=False)

run_config = RunConfig(
    name="ptl-exp-tracking",
    storage_path="/tmp/ray_results",
)

trainer = LightningTrainer(
    lightning_config=lightning_config,
    scaling_config=scaling_config,
    run_config=run_config,
)

trainer.fit()


2023-04-27 19:13:04,277	INFO data_parallel_trainer.py:357 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(pid=78857) /mnt/cluster_storage/pypi/lib/python3.9/site-packages/neptune/common/warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
(pid=78857)   warnings.warn(
(TrainTrainable pid=78857) 2023-04-27 19:13:13,689	INFO data_parallel_trainer.py:357 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(LightningTrainer pid=78857) 2023-04-27 19:13:13,695	INFO data_parallel_trainer.py:357 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enab

Trial name,_report_on,date,done,epoch,experiment_tag,hostname,iterations_since_restore,metric_1,metric_2,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,train_loss,training_iteration,trial_id
LightningTrainer_35598_00000,train_epoch_end,2023-04-27_19-13-48,True,4,0,ip-10-0-74-175,5,30,300,10.0.74.175,78857,True,20,34.5741,0.210718,34.5741,1682648028,0.792705,5,35598_00000


(RayTrainWorker pid=79231) COMET INFO: ---------------------------
(RayTrainWorker pid=79231) COMET INFO: Comet.ml Experiment Summary
(RayTrainWorker pid=79231) COMET INFO: ---------------------------
(RayTrainWorker pid=79231) COMET INFO:   Data:
(RayTrainWorker pid=79231) COMET INFO:     display_summary_level : 1
(RayTrainWorker pid=79231) COMET INFO:     url                   : https://www.comet.com/woshiyyya/demo-project/a98ac061d68d438f8ae605cd4e6ea357
(RayTrainWorker pid=79231) COMET INFO:   Metrics [count] (min, max):
(RayTrainWorker pid=79231) COMET INFO:     metric_1 [20]   : (0.0, 30.0)
(RayTrainWorker pid=79231) COMET INFO:     metric_2 [20]   : (0.0, 300.0)
(RayTrainWorker pid=79231) COMET INFO:     train_loss [20] : (0.6525752544403076, 0.8741906881332397)
(RayTrainWorker pid=79231) COMET INFO:   Others:
(RayTrainWorker pid=79231) COMET INFO:     Name : demo-run
(RayTrainWorker pid=79231) COMET INFO:   Uploads:
(RayTrainWorker pid=79231) COMET INFO:     conda-environment-d

Result(
  metrics={'_report_on': 'train_epoch_end', 'train_loss': 0.7927054166793823, 'metric_1': 30.0, 'metric_2': 300.0, 'epoch': 4, 'step': 20, 'should_checkpoint': True, 'done': True, 'trial_id': '35598_00000', 'experiment_tag': '0'},
  path='/tmp/ray_results/ptl-exp-tracking/LightningTrainer_35598_00000_0_2023-04-27_19-13-04',
  checkpoint=LightningCheckpoint(local_path=/tmp/ray_results/ptl-exp-tracking/LightningTrainer_35598_00000_0_2023-04-27_19-13-04/checkpoint_000004)
)